In [ ]:
# default stuff (display width, dir change, jupyter extentions)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import anodeclstmgru.constants as const
import os
from anodeclstmgru.models.lit_module import AutoEncoderLitModule
from anodeclstmgru.data.data_module import SWaTSDataModule
import yaml
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact
import torch
import numpy as np
from sklearn.metrics import mean_squared_error

In [ ]:
MODEL_VERSION = 1 
hparams_path = f'./lightning_logs/version_{MODEL_VERSION}/hparams.yaml'
with open(hparams_path, 'r') as stream:
        hparam_dct = yaml.safe_load(stream)
ckpt_file_name = os.listdir(f'./lightning_logs/version_{MODEL_VERSION}/checkpoints/')[0]
ckpt_file_path = f'./lightning_logs/version_{MODEL_VERSION}/checkpoints/{ckpt_file_name}'

In [ ]:
dm_train = SWaTSDataModule(**hparam_dct)
dm_train.setup()
train_data_loade = dm_train.train_dataloader()

In [ ]:
batch_in = iter(train_data_loade).next()
df_in = pd.DataFrame(batch_in[0,:,:].numpy(), columns=const.SENSOR_COLS)

In [ ]:
model = AutoEncoderLitModule.load_from_checkpoint(ckpt_file_path)

In [ ]:
batch_out = model(batch_in)
df_out = pd.DataFrame(batch_out[0,:,:].detach().numpy(), columns=const.SENSOR_COLS)

In [ ]:
df_in.head()

In [ ]:
df_out.head()

In [ ]:
def plot_ts_and_reconstruction(signal):
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
    for df, key in zip([df_in, df_out], ['orig.', 'reconstr.']):
        fig.add_trace(go.Scatter(x=df.index, y=df[signal], name=f'{signal}_{key}'), row=1, col=1)
    fig.show()

interact(plot_ts_and_reconstruction, signal=const.SENSOR_COLS)

In [ ]:
# signal = 'LIT101'
# fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
# fig.add_trace(go.Scatter(x=dm_train.swats_train.dataset.df.index,
#                          y=dm_train.swats_train.dataset.df[signal]), row=1, col=1)
# fig.show()

In [ ]:
hparam_att_dct = hparam_dct.copy()
hparam_att_dct.update(dict(normal=False))

In [ ]:
from anodeclstmgru.data.dataset import SWaTSDataset

In [ ]:
ds_test = SWaTSDataset(**hparam_att_dct)
ds_test.df.head()

In [ ]:
MSE_VALS_STEP_SIZE = 100
end_idxs = list(range(ds_test.window_size,
                 len(ds_test.df),
                 MSE_VALS_STEP_SIZE))
timestamps = list(ds_test.df.index[end_idxs])
df_test = df_test[df_test.notna()]
df_test = ds_test.df.reset_index()
input_array = np.stack([df_test.loc[i+1-dm_test.window_size:i][const.SENSOR_COLS].values.astype(np.float32)
                for i in end_idxs])
output_array = model(torch.tensor(input_array))
output_array = output_array.detach().numpy()

In [ ]:
df_test[df_test.notna()]

In [ ]:
input_array_list = [input_array[i, :, :] for i in  range(input_array.shape[0])]
output_array_list = [output_array[i, :, :] for i in  range(output_array.shape[0])]

In [ ]:
mse_list = [mean_squared_error(i, o) for i, o in zip(input_array_list, output_array_list)]

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=timestamps, y=mse_list, name='mse'), row=1, col=1)
fig.show()
